# Peripherals

In [1]:
%connect serial:///dev/ttyAMA1

Connected to robot-stm32 @ serial:///dev/ttyAMA1


## Configuration

`/boot/config.txt`: add

```
[pi4]
dtoverlay=uart3
dtoverlay=uart4
enable_uart=0
```

Balena: configure dtoverlay from the dashboard.

Overlay `enable_uart=0` disables the console UART on pins 8, 10. No need to disable if pins are not connected (but then there is no way this can work with a Pi 3, probably ok).

UART Assignments (Pi 4):

```
Pi 4                      STM32
/dev/ttyAMA1   uart3      uart6 (REPL)
/dev/ttyAMA2   uart4      uart3 (dfu, robot comm)
```

GPIO Pins:

```
BOOT0   = 13 (GPIO27)
SHUT_DN = 33 (GPIO13)     shutdown button input (active low)
nRST    = 40 (GPIO21)
AUX     = 36 (GPIO16)     stm32: AUX
                          stm32: PWR_EN: pull low to cut main (5V) power supply
                          stm32: V12_DIV: 12V input / 7.8 
```

## GPIO

List of available GPIOs and their capabilities:

In [1]:
# pin alternate functions

from pyb import Pin

print("{:15} {:4} {}".format('board name', 'name', 'alternate functions'))
for p in sorted(dir(Pin.board)):
    if '__' in p: continue
    pin = Pin(p)
    names = pin.names()
    print("{:15} {:4} {}".format(names[1], names[0], pin.af_list()))

board name      name alternate functions
AIN1            C5   []
AIN2            C1   []
AUX             B13  [Pin.AF1_TIM1, Pin.AF5_SPI2, Pin.AF7_USART3]
BATTERY_MONITOR A3   [Pin.AF1_TIM2, Pin.AF2_TIM5, Pin.AF3_TIM9, Pin.AF7_USART2]
BIN1            C3   [Pin.AF5_SPI2]
BIN2            A4   [Pin.AF5_SPI1, Pin.AF7_USART2]
BOOT1           B2   []
D8              C0   []
ENC_A1          A6   [Pin.AF1_TIM1, Pin.AF2_TIM3, Pin.AF3_TIM8, Pin.AF5_SPI1, Pin.AF9_TIM13]
ENC_A2          A7   [Pin.AF1_TIM1, Pin.AF2_TIM3, Pin.AF3_TIM8, Pin.AF5_SPI1, Pin.AF9_TIM14]
ENC_B1          B6   [Pin.AF2_TIM4, Pin.AF4_I2C1]
ENC_B2          B7   [Pin.AF2_TIM4, Pin.AF4_I2C1]
FLASH_CS        A15  [Pin.AF1_TIM2, Pin.AF1_TIM2, Pin.AF5_SPI1]
FLASH_MISO      B4   [Pin.AF2_TIM3, Pin.AF5_SPI1]
FLASH_MOSI      B5   [Pin.AF2_TIM3, Pin.AF5_SPI1]
FLASH_SCK       B3   [Pin.AF1_TIM2, Pin.AF5_SPI1]
NC_A0           A0   [Pin.AF1_TIM2, Pin.AF1_TIM2, Pin.AF2_TIM5, Pin.AF3_TIM8, Pin.AF7_USART2]
NC_A1           A1   [Pin.AF1_TIM2,

## Motor Controller

The motor controller comprises two H-bridges to control the speed and direction of the motors and magnetic encoders to capture the rotational speed.

### H-bridges

The figure below shows the TB6612 motor controller. It is wired to the STM32 as follows:

* Channel A control: PWMA, AIN1, AIN2
* Channel B control: PWMB, BIN1, BIN2

```{figure} figures/TB6612-breakout.jpg
:width: 300px
:name: h-bridge

[TB6612](https://www.pololu.com/product/713) dual H-bridge motor controller breakout.
```

The `xIN1` and `xIN2` determine its state as follows:

| Function | IN1 | IN2 | Notes |
|----------|-----|-----|-------|
| Forward  |  L  |  H  |  
| Reverse  |  H  |  L  |
| Brake    |  H  |  H  | speed=0 has same effect
| Coast (Z)|  L  |  L  |

The TB6612 class maps the desired speed input to the appropriate control signals:

In [1]:
%cat /flash/lib/tb6612.py

"""
TB6612 motor driver.

Example:

from pyb import Pin, Timer
from tb6612 import TB6612
import time

freq = 10_000
tim = Timer(8, freq=freq)

motor1 = TB6612(
    tim.channel(3, Timer.PWM_INVERTED, pin=Pin('PWM_A')),
    Pin('AIN1', mode=Pin.OUT_PP),
    Pin('AIN2', mode=Pin.OUT_PP)
)

motor2 = TB6612(
    tim.channel(1, Timer.PWM_INVERTED, pin=Pin('PWM_B')),
    Pin('BIN1', mode=Pin.OUT_PP),
    Pin('BIN2', mode=Pin.OUT_PP)
)

nstby = Pin('NSTBY', mode=Pin.OUT_PP)
nstby.value(1)

motor1.speed(30)
motor2.speed(-83)

time.sleep(5)
nstby.value(0)
"""

class TB6612:
    
    def __init__(self, pwm, scale, in1, in2):
        self.pwm = pwm
        self.scale = scale
        self.in1 = in1
        self.in2 = in2
        
    def speed(self, speed:float):
        """Set motor speed (duty cycle), range +/- 100."""
        if speed < 0:
            self.in1.value(0)
            self.in2.value(1)
        else:
            self.in1.value(1)
            self.in2.value(0)
        speed *= self.sc

Example:

In [1]:
from tb6612 import TB6612
from pyb import Pin, Timer

# motor power control
nstby = Pin('NSTBY', mode=Pin.OUT_PP)
nstby.value(0)

# motors
pwm_timer = Timer(8, freq=10_000)
scale = (pwm_timer.period()+1)/100

motor1 = TB6612(
    pwm_timer.channel(3, Timer.PWM_INVERTED, pin=Pin('PWM_A')),
    scale,
    Pin('AIN1', mode=Pin.OUT_PP),
    Pin('AIN2', mode=Pin.OUT_PP)
)

motor2 = TB6612(
    pwm_timer.channel(1, Timer.PWM_INVERTED, pin=Pin('PWM_B')),
    scale,
    Pin('BIN1', mode=Pin.OUT_PP),
    Pin('BIN2', mode=Pin.OUT_PP)
)

16799 167.99


Start motors: if everything is wired correctly, the motors will spin when running the code in next cell.

In [1]:
from time import sleep

# enable TB6612
nstby.value(1)

for speed in range(0, 101, 20):
    # set desired speed (PWM duty cycle)
    motor1.speed(speed)
    motor2.speed(-speed/2)
    sleep(1)
    
motor1.speed(0)
motor2.speed(0)

### Encoder

Just like a car, motor speed depends on load. Encoders can be used to measure the actual speed.


I've chosen magnetic encoders:

```{figure} figures/encoder.jpg
:width: 300px
:name: h-bridge

[Magnetic Encoder](https://www.pololu.com/product/3081) breakout boards.
```

A wheel with several magnetic zones with with alternating magnetization mounted to the shaft to the motor passes two magnetic sensors (the black components on the board) to produce electrical pulses. The STM32 has built-in counters to tally up these pulses.

In [1]:
!cat code/stm32/lib/encoder.py

# https://github.com/iot49/upy-examples/blob/master/encoder2.py

"""
Hardware quadrature encoder.
Works for Timers 2-5
  2 & 5 have 32-bit counters
  3 & 4 have 16-bit counters
Input is on ch1 & ch2.

Sample usage:

enc1 = init_encoder(3, Pin.cpu.A6, Pin.cpu.A7, Pin.AF2_TIM3)
enc2 = init_encoder(4, Pin.cpu.B6, Pin.cpu.B7, Pin.AF2_TIM4)

Readout (c2, 2's complement, only for 16-bit timers):
c2(enc1.counter())

Reset:
enc1.counter(0)
"""

from pyb import Pin, Timer

def init_encoder(timer, ch1_pin, ch2_pin, af):
    # sets up encoder and returns timer
    # read count by calling timer.counter()
    pin_a = Pin(ch1_pin, Pin.AF_PP, pull=Pin.PULL_NONE, af=af)
    pin_b = Pin(ch2_pin, Pin.AF_PP, pull=Pin.PULL_NONE, af=af)
    enc_timer = Timer(timer, prescaler=0, period=0xffff)
    enc_channel = enc_timer.channel(1, Timer.ENC_AB)
    return enc_timer


def c2(x):
    # two's complement of 16-Bit number
    if x & 0x8000:
        # two's complement of 16 bit value
        x -= 0x10000
    ret

In [1]:
from encoder import init_encoder, c2

enc1 = init_encoder(4, 'ENC_A1', 'ENC_A2', Pin.AF2_TIM4)
enc2 = init_encoder(3, 'ENC_B1', 'ENC_B2', Pin.AF2_TIM3)

A complete example:

In [1]:
import time 

# enable TB6612
nstby.value(1)

for speed in range(0, 101, 20):
    # set desired speed (PWM duty cycle)
    motor1.speed(speed)
    motor2.speed(-speed // 2)
    enc1.counter(0)
    enc2.counter(0)
    time.sleep(0.5)
    cnt1 = c2(enc1.counter())
    cnt2 = c2(enc2.counter())
    ratio = cnt1/cnt2 if cnt2 != 0 else 0
    
    print(f"{cnt1:6d}  {cnt2:6d}   {ratio:6.2f}")
    
nstby.value(0)

     0       0     0.00
  -343       0     0.00
  -952    -389     2.45
 -1567    -715     2.19
 -2194   -1032     2.13
 -2847   -1353     2.10


## IMU

To balance on two wheels, the robot needs to know about the direction of gravity. Specifically, we want to know the pitch angle in the figure below.

```{figure} ../figures/yaw_pitch_roll.png
:width: 300px

Pitch, roll, and heading.
```

Accelerometers and gyroscopes are perfect for this task. We will look at two solutions - one using a BNO055 IMU with on-board motion processor that directly outputs quaternions from which gravity can be calculated. 

The other option uses a complementary filter to extract gravity from raw accelerometer and gyroscope output.

Below is the code using quaternions and the output from flipping the robot over.

In [1]:
from bno055 import BNO055, NDOF_FMC_OFF_MODE, QUAT_DATA
from machine import I2C
from time import sleep_ms
import math

RAD2DEG = 180/math.pi

# init
i2c = I2C(1, freq=400_000)
imu = BNO055(i2c, crystal=True, transpose=(2, 0, 1), sign=(0, 0, 1))
imu.mode(NDOF_FMC_OFF_MODE)
sleep_ms(800)

# read sensor and compute pitch angle
for i in range(10):
    imu.iget(QUAT_DATA)
    w = imu.w/(1<<14)
    x = imu.x/(1<<14)
    y = imu.y/(1<<14)
    z = imu.z/(1<<14)
    sinp = 2 * (w * y - z * x)
    if abs(sinp) >= 1:
        # use 90 degrees if out of range
        pitch = math.copysign(math.pi / 2, sinp)
    else:
        pitch = math.asin(sinp)
    pitch *= RAD2DEG
    print(f"{pitch:8.2f}")
    sleep_ms(500)

  -88.01
  -88.01
  -87.82
  -54.62
  -46.12
  -29.56
    3.47
   42.27
   61.86
   83.53


A little bit optimized:

In [1]:
from bno055 import BNO055, NDOF_FMC_OFF_MODE, QUAT_DATA
from machine import I2C
from time import sleep_ms
import math

_RAD2DEG = 180/math.pi
_90DEG = 0.5 * math.pi
_QUAT_SCALE = 1/(2<<26)

# init
i2c = I2C(1, freq=400_000)
imu = BNO055(i2c, crystal=True, transpose=(2, 0, 1), sign=(0, 0, 1))
imu.mode(NDOF_FMC_OFF_MODE)
sleep_ms(800)

# read sensor and compute pitch angle
for i in range(10):
    imu.iget(QUAT_DATA)
    sinp = _QUAT_SCALE * (imu.w * imu.y - imu.z * imu.x)
    if abs(sinp) >= 1:
        # use 90 degrees if out of range
        pitch = math.copysign(_90DEG, sinp)
    else:
        pitch = math.asin(sinp)
    pitch *= _RAD2DEG
    
    print(f"{pitch:8.2f}")
    sleep_ms(500)

  -88.19
  -88.15
  -42.03
    5.38
   52.51
   75.75
   42.02
   23.82
   46.48
   76.74


## Complementary Filter

The code below reports the computation time (including reading the sensor) and orientation from:

1) accelerometers only
2) gyroscope only
3) complementary filter combining accelerometer and gyroscope data
4) Euler angle computed by BNO055

The third option is robust and ideal for low-cost inertial sensors without on-board processing.

In [1]:
# rotate about short side of chip

import time, math, gc
from machine import I2C as I2C
from pyb import Pin
from bno055 import BNO055

tau = 0.2
Ts = 0.02
ALPHA = tau / (tau+Ts)
RAD_TO_DEG = 180/math.pi

i2c = I2C(1, freq=400_000)

imu = BNO055(i2c)
time.sleep(1)

a_off = 0
g_phi = -90
c_phi = 0

t_last = time.ticks_us()

dt = 0

print("{:>8s} {:>6s} {:>6} {:>6} {:>6}".format("dt [us]", "xcell", "gyro", "comp", "bno055"))
for i in range(1000):
    # xcell
    ax, ay, az = imu.accel()
    a_phi = math.atan2(ay, az)*RAD_TO_DEG - a_off
    if i == 0: a_off = a_phi+90
    
    # gyro
    rx, ry, rz = imu.gyro()
    g_phi += rx * Ts * 0.5  # why 0.5?
    
    # complementary
    c_phi = ALPHA * (c_phi + rx * Ts) + (1-ALPHA) * a_phi
    
    # euler
    _, _, e_phi = imu.euler()
    
    if i % 50 == 0:
        print("{:8d} {:6.0f} {:6.0f} {:6.0f} {:6.0f}".format(int(dt), a_phi, g_phi, c_phi, -e_phi-90))
    
    # clean slate
    gc.collect()

    # sleep off extra time
    t_new = time.ticks_us()
    dt = time.ticks_diff(t_new, t_last)
    t_last = t_new
    time.sleep(Ts-dt*1e-6)


 dt [us]  xcell   gyro   comp bno055
       0     -0    -90     -0    -91
   17999    -90    -90    -89    -91
   10447    -90    -90    -90    -91
    5354   -113   -111   -117   -114
    7995   -132   -131   -137   -134
    5399   -157   -153   -161   -158
    2593   -173   -168   -177   -176
   11001   -197   -187   -199   -196
    9994   -217   -206   -220   -216
   10603   -234   -221   -236   -233
   15108   -243   -231   -245   -244
   11001   -256   -242   -258   -256
    8185     84   -256   -123     88
    3003     90   -256     84     88
    8608     90   -256     90     88
    7747     90   -256     90     88
    5926     90   -256     90     88
    4437     90   -256     90     88
    8183     90   -256     90     88
   10926     90   -256     90     88
